In [1]:
from subtitle_process import fetch_subtitle
import getpass
import os
import json
import re
from templates import *
from functions import inference

In [2]:
# 设置临时环境变量
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("bilibili_ssid")

In [3]:
with open("url.json", 'r') as file:
    data = json.load(file)

In [4]:
data[0]

'BV1FU411f7AX'

In [5]:
problems = []
for bv in data:
    problems.append(fetch_subtitle(bv))

Extracted segment: 1FU411f7AX
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/190453309015469940094d5ad77763724ec3218b62ecfdcaee9d?auth_key=1720585368-ccfeba5022794bfabcdc7438cfacc005-0-48e149ba6e0f4c8d86629ea127119094
Extracted segment: 1Hx4y167B2
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/10036383521516886557971445b1fe1305ef022ff4db56987934?auth_key=1720585373-6349865ef0d24085b19743916a7b0b6d-0-58119b322f7d1e547b0278aa6d8dfa54
Extracted segment: 1Ux4y187nS
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/10033403111512448704033241eea1d30f40e89a16f05f3b79e0?auth_key=1720585377-bd509119df9a4553a77615728467259b-0-782713360e03998436f720c6d61c39af
Extracted segment: 16z4y1w7h2
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/5784052141332564220823b85ecf0

In [6]:
def extract_question_explanations(responses):
    questions_and_explanations = []
    for response in responses:
        pattern = re.compile(r"### 原题：\n(.+?)\n\n### 讲解：\n(.+?)\n\n### 相似题一：\n(.+?)\n\n### 相似题一讲解：\n(.+?)\n\n", re.DOTALL)
        matches = pattern.findall(response)

        for match in matches:
            questions_and_explanations.append({
                "original_question": match[0].strip(),
                "original_explanation": match[1].strip(),
                "similar_question_1": match[2].strip(),
                "similar_explanation_1": match[3].strip(),
            })

    with open('casestudy_problem_answer.json', 'w', encoding='utf-8') as f:
        json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)


In [7]:
template_sub_1_similar = r"""
以下是一段视频字幕，请提取出题目和老师对题目的讲解，同时构造一个相似题目并生成与老师讲解风格相似的讲解。请按如下格式输出：

### 原题：
（提取出的原题）

### 讲解：
（提取出的讲解）

### 相似题一：
（构造的相似题目一）

### 相似题一讲解：
（构造的相似题目一的讲解）

视频字幕：
{subtitle}

注意事项：
1. 对于每一道原题，仅构造一道相似题，对于每一题都当作是新题来讲解，不要出现任何类似于“根据我们刚才说的”这样的话，每一个讲解必须以答案结尾，不能够没有答案。
2. 保留老师讲解风格但是把和讲解无关的内容过滤掉，比如自问自答的部分都要删除掉。
"""

template_expl = r"""
    Role:
        你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一的讲解一个题目。 
    Goals:
        根据给定的板书内容像老师一样讲解板书内容，根据输入决定你正在输出的是讲解几，比如如果输入是板书一，那么输出就是讲解一，以此类推。

    严格按照以下列字符串格式输出:
        ### 讲解一:    ### 讲解二:      ...

    原题和参考信息:
        题目:{question}
        参考信息:{ref}

    注意事项:
        1. 你的讲解要尽量贴合人类老师的讲解风格，言简意赅，清晰明了。
        2. 讲解内容应针对板书的具体步骤进行详细说明，解释每一步的思路和方法。
        3. 确保每一个讲解都对应一个板书。
        4. 板书和讲解的编号必须一一对应，格式必须严格遵守。
        5. 讲解内容应突出解题思路和方法，避免过多冗余信息。
        6. 你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇
        7. 其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题，别用括号1234。
            d: 初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
            e: 对于所有题目尽可能有最简单的方式来进行解答，以最大限度上避免超纲的情况发生。

    下面是板书的内容(请确保每一个讲解都对应一个板书)：
    {notes}

    下面是一些名师讲解的其他题目，请你学习他们的讲解风格，并运用到板书的讲解中(注意：你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇)。
    注意：初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
    {model_expl}
"""

In [8]:
raw_problem_answer = []
for problem in problems:
    raw_problem_answer.append(await inference({"subtitle": problem}, template_sub_1_similar))


executing
### 原题：
两个质数的和为39，这两个质数分别是多少？

### 讲解：
我们假设这两个质数分别是A和B，且A + B = 39。首先，我们需要注意到39是一个奇数。根据数的性质，只有奇数加偶数才能等于奇数。因此，这两个质数中必须有一个是奇数，另一个是偶数。再满足第二个条件，即两个数都要是质数。我们知道唯一的偶质数是2。那么，A或B中有一个是2，另一个数就是37。因此，这两个质数分别是37和2。

### 相似题一：
两个质数的和为41，这两个质数分别是多少？

### 相似题一讲解：
我们假设这两个质数分别是A和B，且A + B = 41。首先，我们需要注意到41是一个奇数。根据数的性质，只有奇数加偶数才能等于奇数。因此，这两个质数中必须有一个是奇数，另一个是偶数。再满足第二个条件，即两个数都要是质数。我们知道唯一的偶质数是2。那么，A或B中有一个是2，另一个数就是41 - 2 = 39。但是39不是质数，所以我们需要重新考虑。我们可以尝试其他组合，最终发现41可以分解为两个质数的和，即5和36，但36不是质数。继续尝试，最终发现41可以分解为两个质数的和，即11和30，但30也不是质数。最后，我们发现41可以分解为两个质数的和，即3和38，但38也不是质数。经过多次尝试，我们发现41不能分解为两个质数的和。因此，这个题目没有符合条件的答案。executing
### 原题：
有五元、十元的人民币共14张，共100元。五元和十元各几张？

### 讲解：
这里总共有14张人民币。我们假设这14张人民币全是五元的，那么总金额就是14×5=70元。但实际总金额是100元，所以这里有一个差额。这个差额是因为我们把十元的人民币当成了五元的人民币。每张十元的人民币比五元的多5元，所以总金额会少掉5元。总共少掉的金额是100-70=30元。我们可以通过这个差额来计算十元的张数。十元的张数就是总共少掉的金额除以每张少掉的金额，即30÷5=6张。所以十元的有6张。五元的张数就是总数14减去十元的张数6，等于8张。

### 相似题一：
有两元、五元的人民币共20张，共60元。两元和五元各几张？

### 相似题一讲解：
这里总共有20张人民币。我们假设这20张人民币全是两元的，那么总金额就是20×2=40元。但实际总金额是60元，所以这里有一个差额。这个差

In [9]:
extract_question_explanations(raw_problem_answer)

In [10]:
with open("casestudy_problem_answer.json", 'r') as file:
    data = json.load(file)

data

[{'original_question': '一个数的小数点向右移动一位比原数就大135，求原数是多少？',
  'original_explanation': '根据题意，小数点向右移动一位，数值会变成原数的十倍。我们可以举例子来看，比如2.5，小数点向右移动一位变成25，0.5向右移动一位变成5，前后对比，变化后的数字是原数的十倍。所有这样的数字都是这样，只要小数点向右移动一位，变化后的数是原数的十倍。\n\n根据题目，我们可以得到变化后的数是原数的十倍。变化后的数比原数多135，这就是差倍问题。我们假设原数是x，那么变化后的数就是10x。根据题意，10x比x多135，所以我们可以列出方程：\n\n10x - x = 135\n\n解这个方程：\n\n9x = 135\n\nx = 135 / 9\n\nx = 15\n\n所以，原数是15。我们可以验算一下，15小数点向右移动一位变成150，150比15大135，验证正确。',
  'similar_question_1': '一个数的小数点向右移动两位比原数就大990，求原数是多少？',
  'similar_explanation_1': '根据题意，小数点向右移动两位，数值会变成原数的100倍。我们可以举例子来看，比如0.25，小数点向右移动两位变成25，0.05向右移动两位变成5，前后对比，变化后的数字是原数的100倍。所有这样的数字都是这样，只要小数点向右移动两位，变化后的数是原数的100倍。'}]

In [11]:
model_explanation = ""
for ps in data[:3]:
    model_explanation += "题目:\n" + ps['original_question'] + '\n\n'
    model_explanation += "讲解:\n" + ps['original_explanation'] + '\n\n'
    # model_explanation += "题目:\n" + ps['similar_question_1'] + '\n\n'
    # model_explanation += "讲解:\n" + ps['similar_explanation_1'] + '\n\n'
model_explanation

'题目:\n一个数的小数点向右移动一位比原数就大135，求原数是多少？\n\n讲解:\n根据题意，小数点向右移动一位，数值会变成原数的十倍。我们可以举例子来看，比如2.5，小数点向右移动一位变成25，0.5向右移动一位变成5，前后对比，变化后的数字是原数的十倍。所有这样的数字都是这样，只要小数点向右移动一位，变化后的数是原数的十倍。\n\n根据题目，我们可以得到变化后的数是原数的十倍。变化后的数比原数多135，这就是差倍问题。我们假设原数是x，那么变化后的数就是10x。根据题意，10x比x多135，所以我们可以列出方程：\n\n10x - x = 135\n\n解这个方程：\n\n9x = 135\n\nx = 135 / 9\n\nx = 15\n\n所以，原数是15。我们可以验算一下，15小数点向右移动一位变成150，150比15大135，验证正确。\n\n'

In [12]:
orig_answers = []
orig_notes = []
orig_expls = []
refined_explanation = []
questions = []

In [13]:
'''
对于每一个<原题,相似题>，用两个prompt来分别对相似题进行解答
    1. 用original prompt对相似题直接进行解答
    2. 将原题解答及相似题解答作为few shots examples喂给另一个prompt来对比两种的效果哪一个比较好
'''

with open("casestudy_problem_answer.json", 'r') as file:
    problem_set = json.load(file)


for ps in problem_set:
    orig_answers.append(await inference({"question": ps['similar_question_1'], "ref_form": ref_form, "ref": ref}, template_orig))
    pattern1 = r"### 板书一:[\s\S]*?(?:### 板书二:[\s\S]*?)?(?=### 讲解一)"
    orig_note = re.search(pattern1, orig_answers[-1]).group()
    pattern2 = r"### 讲解一:[\s\S]*"
    orig_expl = re.search(pattern2, orig_answers[-1]).group()
    orig_notes.append(orig_note)
    orig_expls.append(orig_expl)
    questions.append(ps['similar_question_1'])
    refined_explanation.append(await inference({"question": ps['similar_question_1'], "ref": ref, "notes": orig_note, "model_expl": model_explanation}, template_expl))

# print(orig_notes)
# print(refined_explanation)    

executing
知识点讲解:本题考查的知识点是方程的建立与解法，解题关键点是根据题意建立方程并求解。

### 板书一:
设原数为$x$，
依题意得：
$x \times 100 = x + 990$

### 板书二:
$100x = x + 990$

### 板书三:
$100x - x = 990$

### 板书四:
$99x = 990$

### 板书五:
$x = \frac{990}{99}$

### 板书六:
$x = 10$

故答案是：原数是10。

### 讲解一:
首先设原数为$x$，根据题意，原数的小数点向右移动两位相当于乘以100，因此可以得到方程$x \times 100 = x + 990$。

### 讲解二:
将方程整理为$100x = x + 990$，然后将$x$移到方程的一边，得到$100x - x = 990$。

### 讲解三:
合并同类项，得到$99x = 990$。

### 讲解四:
将方程两边同时除以99，得到$x = \frac{990}{99}$。

### 讲解五:
计算得出$x = 10$。

### 讲解六:
因此，原数是10。故答案是：原数是10。executing
### 讲解一:
首先，我们设原数为$x$。根据题意，小数点向右移动两位意味着这个数会变成原数的100倍。题目告诉我们，这个新数比原数大990。因此，我们可以列出方程：
\[ x \times 100 = x + 990 \]
这一步是为了将题目中的文字描述转化为数学表达式。

### 讲解二:
接下来，我们简化方程。将方程左边的$x \times 100$写成$100x$，这样方程变为：
\[ 100x = x + 990 \]
这一步是为了让方程看起来更简洁，便于后续的计算。

### 讲解三:
为了进一步简化方程，我们将$x$从方程的右边移到左边。这样我们就得到了：
\[ 100x - x = 990 \]
这一步是通过移项来将所有含有$x$的项放在方程的一边，方便求解。

### 讲解四:
接下来，我们将左边的$100x - x$合并成$99x$，方程变为：
\[ 99x = 990 \]
这一步是将同类项合并，进一步简化方程。

### 讲解五:
为了求出$x$的值，我们将方程两边同时除以99，得到：
\[ x = 

In [14]:
with open("compare_results_p5.md", 'a') as file:
    for i in range(len(questions)):
        file.write("# 原题：\n" + questions[i] + '\n\n')
        file.write("# 原本板书：\n" + orig_notes[i] + '\n\n')
        file.write("# 原本解答：\n" + orig_expls[i] + '\n\n')
        file.write("# 新解答：\n" + refined_explanation[i] + '\n\n')

In [31]:
print(len(orig_expls))

12


In [32]:
import re

text = """
知识点讲解:本题考查的知识点是因式分解与方程求解，解题关键点是利用平方差公式将原式化简并求解。

###板书一:
解：
$(2a+2b+1)(2a+2b-1)=63$

$设x=2a+2b$

$则(x+1)(x-1)=63$

$x^2-1=63$

$x^2=64$

$x=±8$

$2a+2b=8$

$a+b=4$

$2a+2b=-8$

$a+b=-4$

故答案是$a+b=4$或$a+b=-4$

###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。
"""

pattern = r"###讲解一:[\s\S]*"
match = re.search(pattern, text).group()

if match:
    print(match)
else:
    print("No match found")


###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。

